In [13]:
from pathlib import Path
from mri_data import utils
import nibabel as nib
import os
import json
import re
import statistics
import numpy as np
import pyperclip
import subprocess
import pandas as pd

In [15]:
dataroot = Path("/media/smbshare/3Tpioneer_bids")
training_work_home = Path("/home/srs-9/Projects/prl_project/training_work_dirs")
work_dir = training_work_home / "test_train0_swinunet" #! edit this
ensemble_dir = work_dir / "ensemble_output"
datalist_file_src = "/home/srs-9/Projects/prl_project/training/test_train0/datalist.json"
datalist_file = work_dir / "datalist.json"

prl_info = pd.read_csv("/home/srs-9/Projects/prl_project/data/PRL_labels.csv")

def subid_from_subject(subject):
    return int(re.search(r"(\d{4})", subject)[1])
prl_info['subid'] = prl_info['ID'].map(subid_from_subject)
prl_info.set_index("subid", inplace=True)

In [5]:
with open(datalist_file, 'r') as f:
    datalist = json.load(f)

subject_paths = {}
for item in datalist['training']:
    label = str(item['label'])
    assert os.path.exists(label)
    subject = re.search(r"(sub-ms\d{4})", label)[1]
    subid = re.search(r"(\d{4})", subject)[1]
    subject_paths[subid] = {}
    subject_paths[subid]['label'] = label
    session = re.search(r"(ses-\d{8})", label)[1]
    subject_paths[subid]['image_dir'] = dataroot / subject / session

In [17]:
subid = 1186
image_dir = subject_paths[str(subid)]['image_dir']
images = ["flair.nii.gz", "phase.nii.gz", "t1.nii.gz"]
image_paths = [image_dir / im for im in images]

label_paths = [subject_paths[str(subid)]['label']]

cmd = utils.open_itksnap_workspace_cmd(image_paths, label_paths, win=False)
print(cmd)
print("TOTAL PRL:", prl_info.loc[subid, 'Total PRL'])
pyperclip.copy(cmd)
subprocess.Popen(cmd.split(" "))

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1186/ses-20170517/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1186/ses-20170517/phase.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1186/ses-20170517/t1.nii.gz -s /media/smbshare/3Tpioneer_bids/sub-ms1186/ses-20170517/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz
TOTAL PRL: 1


<Popen: returncode: None args: ['itksnap', '-g', '/media/smbshare/3Tpioneer_...>

Launching ITK-SNAP


In [22]:
target_dataroot = Path("/media/srs-9/Data/3Tpioneer_bids")
with open("copy_cmds.sh", 'w') as f:
    for item in datalist['training']:
        subject = re.search(r"(sub-ms\d{4})", label)[1]
        session = re.search(r"(ses-\d{8})", label)[1]
        base_dir = dataroot / subject / session
        lesion_dir = base_dir / "lesion.t3m20"
        assert lesion_dir.exists()
        target_dir = target_dataroot / subject / session / "lesion.t3m20"
        if not target_dir.exists():
            cmd = f"cp -r {lesion_dir} {target_dir}"
            f.write(cmd + "\n")

    # for item in datalist['testing']:
    #     subject = re.search(r"(sub-ms\d{4})", label)[1]
    #     session = re.search(r"(ses-\d{8})", label)[1]
    #     base_dir = dataroot / subject / session
    #     lesion_dir = base_dir / "lesion.t3m20"
    #     assert lesion_dir.exists()
    #     target_dir = target_dataroot / subject / session / "lesion.t3m20"
    #     if not target_dir.exists():
    #         cmd = f"cp -r {lesion_dir} {target_dir}"
    #         f.write(cmd + "\n")

In [29]:
target_dataroot = Path("/media/srs-9/Data/3Tpioneer_bids")
with open("copy_cmds.sh", 'w') as f:
    for _,row in prl_info.iterrows():
        subject = f"sub-{row['ID']}"
        session = f"ses-{row['date_mri']}"
        base_dir = dataroot / subject / session
        lesion_dir = base_dir / "lesion.t3m20"
        assert lesion_dir.exists()
        target_dir = target_dataroot / subject / session / "lesion.t3m20"
        if not target_dir.exists():
            cmd = f"cp -r {lesion_dir} {target_dir}"
            f.write(cmd + "\n")

In [27]:
lesion_dir

PosixPath('/media/smbshare/3Tpioneer_bids/sub-ms1002/ses-5/21/2020/lesion.t3m20')